In [25]:
import math
from cmath import phase 
import sympy as sym
from numpy import exp, abs, angle, conjugate, sqrt
import numpy as np
from scipy.constants import mu_0, epsilon_0
from scipy.special import k1,k0,i1,i0
from mpmath import *
mp.dps = 25; mp.pretty = True

In [26]:
#Dados dos condutores de fase
#Name: (r0,r1,pfase) ohms/m
CondutoresEspecs = {"Bluejay":(8.702*10**-3,15.977*10**-3,29.544*10**-9), 
                             "Rail":(8.702*10**-3,14.796*10**-3,29.538*10**-9),
                             "Ruddy":(7.821*10**-3,14.364*10**-3,29.559*10**-9),
                             "Grossbeak":(7.456*10**-3,12.573*10**-3,29.538*10**-9),
                             "Dove":(6.988*10**-3,11.773*10**-3,29.567*10**-9),
                             "Penguim":(4.123*10**-3,7.150*10**-3,28.554*10**-9),
                             "Leghorn":(4.840*10**-3,6.7180*10**-3,29.586*10**-9),
                             "Minorca":(4.390*10**-3,6.0960*10**-3,29.579*10**-9),
                             "3/8 EHS":(0,4.570*10**-3,276.470*10**-9)}

In [27]:
#Name: (db,(Xc-1,Xc0,Xc1),(Yc-1,Yc,Yc+1),(Xpr,Ypr,Xpr2,Ypr2),n)
#n é o numero de condutores por fase
#db é o lado do quadrado, quando os condutores estão dispostos como tal
#Xc cordenadas dos centros dos 4 condutores
#Yc cordenadas dos centros dos 4 condutores PS: na formula subtrai por 2/3 da flecha
CondutoresPos = {"Bluejay":(.475,(-15.85,0,15.85),(35.9-2*20.9/3,35.9-2*20.9/3,35.9-2*20.9/3),(-14.45,45.9-2*14.7/3,14.45,45.9-2*14.7/3),4), 
                 "Rail Normal":(.475,(-15,-11,-6,6,11,15),(23.2,33.2,23.2,23.2,33.2,23.2),(-8.8,42.7,8.8,42.7),4),
                    }

#===== 500kV rail convencional, 2 ultimos elementos sao pararraio, da forma (x, y) =====
n_rail_comp = 3
R_rail_comp = (CondutoresEspecs["Rail"][1]*np.sqrt(3))/3 + CondutoresEspecs["Rail"][1]

xy = ((-11.000, 10.736), (-10.771, 11.132), 
       (-11.229, 11.132), (0.000, 10.736), (0.229, 11.132), (-0.229, 11.132), (11.000, 10.736), 
       (10.771, 11.132), (11.229, 11.132), (-6.000, 22.000), (6.000, 22.000))

#centro dos condutores equivalentes (os dois ultimos pares sao dos para-raios)
xyc = ( ( (-10.771 + -11.229)/2 , (11.132 - 2*10.736)/3),
        ( (0.229 + -0.229)/2, (11.132 - 2*10.736)/3),
        ( (10.771 + 11.229)/2, (11.132 - 2*10.736)/3),
        (-6.000, 22.000), (6.000, 22.000) )

R = sqrt((xyc[0][0] - xy[0][0])**2 + (xyc[0][1] - xy[0][1])**2)

CondutoresPos["Rail Convencional"] = (0 , (xyc[0][0],xyc[1][0],xyc[2][0]),
                                      (xyc[0][1],xyc[1][1],xyc[2][1]),
                                      (xyc[3][0],xyc[3][1],xyc[4][0],xyc[4][1]), 3)

#==== 500kV rail compacto, 2 ultimos elementos sao pararraio, da forma (x, y) ====
n_rail_comp = 4
R_rail_comp = 0 

xy2 = ((-4.271, 10.771), (-4.271, 11.229), (-4.729, 11.229), (-4.729, 
   10.771), (0.229, 15.271),
  (0.229, 15.729), (-0.229, 15.729), (-0.229, 15.271), (4.271, 
   10.771), (4.271, 11.229), (4.729, 11.229), (4.729, 
   10.771), (-3.500, 26.000), (3.500, 26.000))

#centro dos condutores equivalentes (os dois ultimos pares sao dos para-raios)
xy2c = ( ((-4.271 + -4.729)/2 , (10.771+11.229)/2),
         ( (0.229 + -0.229)/2, (15.729 + 15.271)/2 ),
         ( (4.271 + 4.729)/2 , (11.229 + 10.771)/2 ),
         (-3.500, 26.000), (3.500, 26.000))

R2 = sqrt((xy2c[0][0] - xy2[0][0])**2 + (xy2c[0][1] - xy2[0][1])**2)

#===== 500kV rail recapacitado, 2 ultimos elementos sao pararraio, da forma (x, y) =====
n_rail_cap = 4
R_rail_cap1 = 0
R_rail_cap2 = 0

#raio do condutor equivalente
xy3 = ((-7.229, 8.639), (-7.229, 10.500), (-6.771, 10.500), (-6.771, 
   8.639), (-0.229, 10.500),
  (-0.229, 9.876), (0.229, 9.876), (0.229, 10.500), (7.229, 
   8.639), (7.229, 10.500), (6.771, 10.500),
  (6.771, 8.639), (-5.000, 20.500), (5.000, 20.500))

xy3c = ( ((-7.229 + -6.771)/2 , (8.639 + 10.500)/2 ),
         ( ((-0.229 + 0.229)/2 , (9.876 + 10.500)/2 ) ),
         ( ((6.771 + 7.229)/2 , (8.639 + 10.500)/2 )),
        (-5.000, 20.500),(5.000, 20.500)) 

R3 = sqrt((xy3c[0][0] - xy3[0][0])**2 + (xy3c[0][1] - xy3[0][1])**2)

#345kv ruddy, 2 ultimos elementos sao pararraio, da forma (x, y)
n_ruddy = 2
#raio do condutor equivalente é igual ao diametro
R_ruddy = 2*CondutoresEspecs["Ruddy"][1]
xy1 = ((-7.229, 10.5), (-6.771, 10.500), (-0.229, 10.500), (0.229, 
   10.500), (7.229, 10.500),
  (6.771, 10.500), (-5.000, 20.500), (5.000, 20.500))

xy1c = ( ((-7.229 + -6.771)/2 , 10.5),
         ((-0.229 + 0.229)/2  , 10.5),
         ((7.229 + 6.771)/2   , 10.5),
        (-5.000, 20.500),(5.000, 20.500))

R1 = sqrt((xy1c[0][0] - xy1[0][0])**2 + (xy1c[0][1] - xy1[0][1])**2)

In [28]:
#função para calcular o raio equivalente
def raio_eq(n, rext, R):
  re = (rext*n*(R**(n-1)))**(n)
  return re

#potencia caracteristica
def Pnat(Vs, Vr, Ir):
  Zc = Vr/Ir
  Pnat = (Vs**2)/Zc.real
  return Pnat

In [35]:
import numpy as np

f=60
omega=2*pi*f
epsilon_r = 10
sigma_s = 1*10**(-3)


def S1(xc,npr,yc,rf,rpr):
#Calcula a matriz de impedancia de retorno do solo
    ncond=len(xc)
    s1=np.eye(ncond)*1j
    for i in range(ncond):
        for j in range(ncond):
            if(i != j):
                s1[i,j] = log(1 + (2/(eta*np.sqrt((xc[i] - xc[j])**2 + (yc[i]+yc[j])**2))))
            elif (i+1) <= nc-npr:
                s1[i,j] = log((1+ 2/(eta*np.sqrt(4*yc[i]**2 + rf**2))))
            else:
                s1[i,j] = log((1+ 2/(eta*np.sqrt(4*yc[i]**2 + rpr**2))))
    return s1

def Mpot(xc,yc,npr,rf,rpr):
  #Calcula a matriz dos potenciais
  ncond=len(xc)
  pot=np.eye(ncond)*1j
  for i in range(ncond):
      for j in range(ncond): 
          if i!=j: #entre fases
              num=(xc[i]-xc[j])**2+(yc[i]+yc[j])**2
              den=(xc[i]-xc[j])**2+(yc[i]-yc[j])**2
              pot[i,j] = 0.5*log(num/den);
          elif (i+1)<= (ncond-npr): #condutor de fase
              pot[i,j] = math.log((2*(yc[i])/rf))
          else: #para-raio
              pot[i,j] = log((2*(yc[i])/rpr))
  return pot

def zexternal(rho,npr,xc,yc,rf,rpr):
        p = math.sqrt(rho/(1j*omega*mu))
        ncond= len(xc)
        zout=np.eye(ncond)*1j
        for i in range(ncond):
            for j in range(ncond): 
                if i!=j: #entre fases
                    num = (xc[i]-xc[j]**2)+(2*p+yc[i]+yc[j]**2)
                    den = (xc[i]-xc[j]**2)+(yc[i]-yc[j]**2)
                    zout[i,j] = 1j*omega*mu_0/(4*pi)*log(num/den)
                elif (i+1)<= (ncond-npr):
                    zout[i,j]= 1j*omega*mu_0/(2*pi)*log((2*(yc[i]+p))/rf)
                else:
                    zout[i,j]= 1j*omega*mu_0/(2*pi)*log((2*(yc[i]+p))/rpr)
        return zout


def Zint(omega,rhoc,rf):
#Calcula a impedancia interna de um condutor cilindrico
    etac = np.sqrt((1j*omega*mu_0)/rhoc)

    zint = rhoc*etac/(2*math.pi*rf)*besseli(0,etac*rf)/besseli(1,etac*rf)
    return zint

def Zinttub(omega,rhoc,rf,ri):
#Calcula a impedancia interna de um condutor tubular
    etac = np.sqrt((1j*omega*mu_0)/rhoc)

    num =  besseli(0,etac*rf)*besselk(1,etac*ri) + besselk(0,etac*rf)*besseli(1,etac*ri) 

    den =  besseli(1, etac*rf)*besselk(1,etac*ri) - besseli(1,etac*ri)*besselk(1,etac*rf)

    zin = rhoc*etac/(2*math.pi*rf)*num/den
    return zin

def Zin(xc,npr,rhoc,rhoc_pr,rf,ri):
    ncond = len(xc);
    zin=np.eye(ncond)*1j
    rpr = CondutoresEspecs["3/8 EHS"][1]

    for i in range(ncond):
        for j in range(ncond): 
            if i!=j: #entre fases
                zin[i,j]=0
            elif (i+1)<= (ncond-npr): #cabos de fase
                  zin[i,j]=Zinttub(omega,rhoc,rf,ri)
            else: #pararaio
                zin[i,j]=Zint(omega,rhoc_pr,rpr)
    return zin

In [38]:
#xyc: posicao dos condutores, ja foi calculado na parte "Convencional - Rail"
xc = (xyc[0][0],xyc[1][0],xyc[2][0],xyc[3][0],xyc[4][0])
print(xc)
yc = (xyc[0][1],xyc[1][1],xyc[2][1],xyc[3][1],xyc[4][1])
print(yc)
npr = 2
rhoc = CondutoresEspecs["Rail"][2]
print(rhoc)
rhoc_pr = CondutoresEspecs["3/8 EHS"][2]
rext_eq = CondutoresEspecs["Rail"][1]
rint_eq = CondutoresEspecs["Rail"][0]
#R: raio da circuinferencia, ja foi calculado na parte "Convencional - Rail"
rf = raio_eq(3, rext_eq, R)
ri = raio_eq(3, rint_eq, R)

Z = Zin(xc,npr,rhoc,rhoc_pr,rf,ri) + (((1j*omega*mu_0)/2/math.pi) * (Mpot(xc,yc,npr,rf,rpr) + S1(xc,npr,yc,rf,rpr)))

(-11.0, 0.0, 11.0, -6.0, 6.0)
(-3.446666666666667, -3.446666666666667, -3.446666666666667, 22.0, 22.0)
2.9538000000000003e-08


TypeError: loop of ufunc does not support argument 0 of type mpc which has no callable sqrt method

In [ ]:
from mpmath import pi

class Linha_transmissao:
    def __init__(self, epsilon_r, sigma_s, r_int, r_ext, nfase, npr, xc, yc, rhoc, rhoc_pr):
        self.f = 60
        self.omega = 2*pi*f
        self.epsilon_r = epsilon_r
        self.sigma_s = sigma_s
        self.r_int = r_int
        self.r_ext = r_ext
        self.nfase = nfase
        self.npr = npr
        self.xc = []
        self.yc = []
        self.rhoc = rhoc
        self.rhoc_pr = rhoc_pr
   
    #Calcula a matriz de impedancia de retorno do solo
    def S1(self, xc,npr,yc,rf,rpr):
        ncond=len(xc)
        s1=np.eye(ncond)*1j
        for i in range(ncond):
            for j in range(ncond):
                if(i != j):
                    s1[i,j] = log(1 + (2/(eta*np.sqrt((xc[i] - xc[j])**2 + (yc[i]+yc[j])**2))))
                elif (i+1) <= nc-npr:
                    s1[i,j] = log((1+ 2/(eta*np.sqrt(4*yc[i]**2 + rf**2))))
                else:
                    s1[i,j] = log((1+ 2/(eta*np.sqrt(4*yc[i]**2 + rpr**2))))
        return s1
    
    
    
    def Mpot(xc,yc,npr,rf,rpr):
    #Calcula a matriz dos potenciais
    ncond=len(xc)
    pot=np.eye(ncond)*1j
      for i in range(ncond):
            for j in range(ncond): 
                if i!=j: #entre fases
                    num=(xc[i]-xc[j])**2+(yc[i]+yc[j])**2
                    den=(xc[i]-xc[j])**2+(yc[i]-yc[j])**2
                    pot[i,j] = 0.5*log(num/den);
                elif (i+1)<= (ncond-npr): #condutor de fase
                    pot[i,j] = math.log((2*(yc[i])/rf))
                else: #para-raio
                    pot[i,j] = log((2*(yc[i])/rpr))
    return pot

    def zexternal(rho,npr,xc,yc,rf,rpr):
        p = math.sqrt(rho/(1j*omega*mu))
        ncond= len(xc)
        zout=np.eye(ncond)*1j
        for i in range(ncond):
            for j in range(ncond): 
                if i!=j: #entre fases
                    num = (xc[i]-xc[j]**2)+(2*p+yc[i]+yc[j]**2)
                    den = (xc[i]-xc[j]**2)+(yc[i]-yc[j]**2)
                    zout[i,j] = 1j*omega*mu_0/(4*pi)*log(num/den)
                elif (i+1)<= (ncond-npr):
                    zout[i,j]= 1j*omega*mu_0/(2*pi)*log((2*(yc[i]+p))/rf)
                else:
                    zout[i,j]= 1j*omega*mu_0/(2*pi)*log((2*(yc[i]+p))/rpr)
        return zout
    
    def Zint(omega,rhoc,rf):
#Calcula a impedancia interna de um condutor cilindrico
    etac = np.sqrt((1j*omega*mu_0)/rhoc)

    zint = rhoc*etac/(2*math.pi*rf)*besseli(0,etac*rf)/besseli(1,etac*rf)
    return zint

def Zinttub(omega,rhoc,rf,ri):
#Calcula a impedancia interna de um condutor tubular
    etac = np.sqrt((1j*omega*mu_0)/rhoc)

    num =  besseli(0,etac*rf)*besselk(1,etac*ri) + besselk(0,etac*rf)*besseli(1,etac*ri) 

    den =  besseli(1, etac*rf)*besselk(1,etac*ri) - besseli(1,etac*ri)*besselk(1,etac*rf)

    zin = rhoc*etac/(2*math.pi*rf)*num/den
    return zin

        
    def Zin(xc,npr,rhoc,rhoc_pr,rf,ri):
    ncond = len(xc);
    zin=np.eye(ncond)*1j
    rpr = CondutoresEspecs["3/8 EHS"][1]

    for i in range(ncond):
        for j in range(ncond): 
            if i!=j: #entre fases
                zin[i,j]=0
            elif (i+1)<= (ncond-npr): #cabos de fase
                  zin[i,j]=Zinttub(omega,rhoc,rf,ri)
            else: #pararaio
                zin[i,j]=Zint(omega,rhoc_pr,rpr)
    return zin    